In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
#web crawling
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
from datetime import date
datestr = date.today().strftime("%d-%m-%Y")

## Crawl for List

In [617]:
driver = webdriver.Firefox(executable_path='/Users/21chang/Desktop/Coding/geckodriver') #!! Change for user
url = 'https://signal.nfx.com/investor-lists/top-saas-seed-investors'

driver.get(url)

# get number of repetitions
header = driver.find_element_by_tag_name("h1").get_attribute('innerHTML')
reps = int(int(header[header.find('<!-- --> (') : header.find(')')].replace('<!-- --> (',""))/8)

# click load more until all is loaded
for i in range(min(reps, 125)):
    loadMore = driver.find_elements_by_xpath("//button[contains(text(),'LOAD MORE INVESTORS')]")
    loadMore[0].click()
    driver.implicitly_wait(10)

# get content
content = driver.find_element_by_css_selector(".w-100").get_attribute('innerHTML')

driver.close()

In [618]:
# writing content to a file
text_file = open("list"+datestr+".txt", "w")
text_file.write(content)
text_file.close()

In [17]:
def findInstances(pattern, string):
    '''
    findInstances: find all indices of a pattern in a string
    '''
    i = string.find(pattern)
    while i != -1:
        yield i
        i = string.find(pattern, i+1)

def remove_tags(html):
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
  
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
  
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [ ]:
'''
Obsolete helper functions
def targetAndRange(text, index):
    [target, ran] = text[text.find('<td class="text-center pt2"><div class="flex-column "><div>', index) : 
                                text.find('</div></div></td><td style="max-width: 400px;"><div><div style="position: relative;"><div class="sn-clamp" style="font-size: 12px; line-height: 1.4;', index)
                               ].replace("<td class=\"text-center pt2\"><div class=\"flex-column \"><div>","").replace("(","").replace(")","").split("</div><div class=\"\">",1)
    #[low, high] = ran.split(" - ")
    return target, ran #low, high
'''

## Crawl for Socials

In [5]:
# !read string from saved file
with open('list'+'12-02-2022'+'.txt', 'r') as file:
    content = file.read().rstrip()


In [6]:
# Initialize masterSocialList depending on running for first time or continuing

#masterSocialList = []

#socialList = pd.read_csv('social'+'12-02-2022'+'.csv')
#socialList = socialList.values.tolist()
#masterSocialList = socialList

In [7]:
urlPrefix = "https://signal.nfx.com"
investorLinks = []
for i in findInstances("href=\"/investors/", content):
    investorLinks.append(urlPrefix + content[i : content.find("\">",i)].replace("href=\"",""))
investorLinks = list(dict.fromkeys(investorLinks))

def getSocials(text):
    '''
    getSocials: returns social media links given the html of an investor profile
    '''
    socialList=[]
    for i in findInstances("href=\"", text):
        socialList.append(text[i : text.find("\" class",i)].replace("href=\"",""))
    return socialList

In [21]:
import undetected_chromedriver.v2 as uc

# Notes on use:
# May time out. If so, manually adjust starting index and re-run code

for link in range(0, len(investorLinks)):
    driver = uc.Chrome()
    driver.get(investorLinks[link])
    driver.implicitly_wait(60)
    
    # search, filter, and append socials
    try:
        masterSocialList.append(getSocials(driver.find_element_by_class_name("sn-linkset").get_attribute('innerHTML')))
    except:
        masterSocialList.append('')
    
    driver.quit()


<ipython-input-21-e966a3c50e4d>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  masterSocialList.append(getSocials(driver.find_element_by_class_name("sn-linkset").get_attribute('innerHTML')))


In [18]:
#newSocialList = []

#for i in masterSocialList:
#    newSocialList.append(i[1])
    
#newSocialList

## Parse into DataFrame

In [ ]:
'''
# Obsolete content parser

nameList=[]
targetList=[]
rangeList = []
#lowList=[]
#highList=[]
locList=[]
fieldList=[]

for i in findInstances("<img alt=\"Photo of ", content):
    nameList.append(content[content.find("<strong class=\"sn-investor-name null\">",i) : content.find("</strong>",i)].replace('<strong class="sn-investor-name null">',""))
    # fix difficulties in finding range
    target, ran = targetAndRange(content, i) #low, high
    targetList.append(target)
    rangeList.append(ran.split(" - "))
    #lowList.append(low)
    #highList.append(high)
    locList.append(remove_tags(content[content.find('<td style="max-width: 400px;"><div><div style="position: relative;"><div class="sn-clamp" style="font-size: 12px; line-height: 1.4; max-height: 50.4px; -webkit-line-clamp: 3; padding-right: 20px;"><iframe style="display: block; opacity: 0; position: absolute; top: 0px; left: 0px; height: 100%; width: 100%; overflow: hidden; pointer-events: none; z-index: -1;" src="about:blank" aria-hidden="true" aria-label="resize-listener" tabindex="-1" frameborder="0"></iframe><div style="position: relative;"><span>',i) : content.find('</a></span></div></div>',i)].replace('<td style="max-width: 400px;"><div><div style="position: relative;"><div class="sn-clamp" style="font-size: 12px; line-height: 1.4; max-height: 50.4px; -webkit-line-clamp: 3; padding-right: 20px;"><iframe style="display: block; opacity: 0; position: absolute; top: 0px; left: 0px; height: 100%; width: 100%; overflow: hidden; pointer-events: none; z-index: -1;" src="about:blank" aria-hidden="true" aria-label="resize-listener" tabindex="-1" frameborder="0"></iframe><div style="position: relative;"><span>',"")))
    fieldList.append(remove_tags(content[content.find('<td style="max-width: 400px;" class=""><div><div style="position: relative;"><div class="sn-clamp" style="font-size: 12px; line-height: 1.4; max-height: 50.4px; -webkit-line-clamp: 3; padding-right: 20px;"><iframe style="display: block; opacity: 0; position: absolute; top: 0px; left: 0px; height: 100%; width: 100%; overflow: hidden; pointer-events: none; z-index: -1;" src="about:blank" aria-hidden="true" aria-label="resize-listener" tabindex="-1" frameborder="0"></iframe><div style="position: relative;"><span>',i) : content.find('</span></div></div></td></tr>',i)].replace('<td style="max-width: 400px;" class=""><div><div style="position: relative;"><div class="sn-clamp" style="font-size: 12px; line-height: 1.4; max-height: 50.4px; -webkit-line-clamp: 3; padding-right: 20px;"><iframe style="display: block; opacity: 0; position: absolute; top: 0px; left: 0px; height: 100%; width: 100%; overflow: hidden; pointer-events: none; z-index: -1;" src="about:blank" aria-hidden="true" aria-label="resize-listener" tabindex="-1" frameborder="0"></iframe><div style="position: relative;"><span>',"")))
    
masterList = np.transpose([nameList, targetList, rangeList, locList, fieldList])
'''

In [137]:
def stripCard(text):
    '''
    stripCard: takes in the plaintext of an investor card (html removed) and returns a list of relevant elements
    '''
    tempList = text.split("Investors in")
    # Handling interest fields
    for i in range(1, len(tempList)-1):
        tempList[i] = tempList[i][0: tempList[i].find("+")]
    tempList[-1] = tempList[-1][0: tempList[-1].find("+")]
    tempList[1:] = [''.join(tempList[1:]).replace("(Other Lists)","")]
    
    # splits numbers from name
    tempList[0] = tempList[0].split(" ")
    tempList.insert(1, tempList[0][tempList[0].index('n/a')+1]) 
    tempList.insert(2, tempList[0][tempList[0].index('n/a')+2].replace("(",""))
    tempList.insert(3, tempList[0][tempList[0].index('n/a')+4].replace(")",""))

    # placeholders for name
    tempList[0] = ""
    tempList.insert(1,"")
    
    return tempList

In [138]:
stripCard(remove_tags(content[ invIndex[0] : invIndex[1]-1 ]))

['Jeff Clavier Uncork Capital Managing Partner n/a $1.2M ($750K - $2.0M) ', ' Palo Alto (CA) , San Francisco (CA) , SF Bay Area (CA)  AI , Cloud Infrastructure , Developer Tools , Hardware , Health IT , IoT , Marketplaces , SaaS (Seed) , San Francisco Bay Area  , Investors who invested in female founders  ']
 Palo Alto (CA


['',
 '',
 '$1.2M',
 '$750K',
 '$2.0M',
 ' Palo Alto (CA) , San Francisco (CA) , SF Bay Area (CA)  AI , Cloud Infrastructure , Developer Tools , Hardware , Health IT , IoT , Marketplaces , SaaS (Seed) , San Francisco Bay Area  , Investors who invested in female founders  ']

In [15]:
from nameparser import HumanName
# https://github.com/derek73/python-nameparser
# LGPL 2.1+

In [143]:
investors = []
nameList = []
locList=[]

invIndex = list(findInstances("<img alt=\"Photo of ", content))
for i in range(1,len(invIndex)):
    
    investors.append(stripCard(remove_tags(content[ invIndex[i-1] : invIndex[i]-1 ])))
    
    name = HumanName(content[content.find("<strong class=\"sn-investor-name null\">",invIndex[i-1]): content.find("</strong>",invIndex[i-1])].replace('<strong class="sn-investor-name null">',""))
    nameList.append([name.first, name.last])
    
    locList.append(remove_tags(content[content.find('<td style="max-width: 400px;"><div><div style="position: relative;"><div class="sn-clamp" style="font-size: 12px; line-height: 1.4; max-height: 50.4px; -webkit-line-clamp: 3; padding-right: 20px;"><iframe style="display: block; opacity: 0; position: absolute; top: 0px; left: 0px; height: 100%; width: 100%; overflow: hidden; pointer-events: none; z-index: -1;" src="about:blank" aria-hidden="true" aria-label="resize-listener" tabindex="-1" frameborder="0"></iframe><div style="position: relative;"><span>',i) : content.find('</a></span></div></div>',i)].replace('<td style="max-width: 400px;"><div><div style="position: relative;"><div class="sn-clamp" style="font-size: 12px; line-height: 1.4; max-height: 50.4px; -webkit-line-clamp: 3; padding-right: 20px;"><iframe style="display: block; opacity: 0; position: absolute; top: 0px; left: 0px; height: 100%; width: 100%; overflow: hidden; pointer-events: none; z-index: -1;" src="about:blank" aria-hidden="true" aria-label="resize-listener" tabindex="-1" frameborder="0"></iframe><div style="position: relative;"><span>',"").replace("Investors in ", "")))
    

['Jeff Clavier Uncork Capital Managing Partner n/a $1.2M ($750K - $2.0M) ', ' Palo Alto (CA) , San Francisco (CA) , SF Bay Area (CA)  AI , Cloud Infrastructure , Developer Tools , Hardware , Health IT , IoT , Marketplaces , SaaS (Seed) , San Francisco Bay Area  , Investors who invested in female founders  ']
 Palo Alto (CA
['Satya Patel Homebrew General Partner n/a $1M ($200K - $2.0M) ', ' Latin America, Canada , United States, San Francisco (CA)  BioTech , BioTech , FinTech , FinTech , Marketplaces , Marketplaces , SaaS (Pre-Seed) , SaaS (Seed) , San Francisco Bay Area  ']
 Latin America, Canada , United States, San Francisco (CA
['Dave Samuel Freestyle Capital General Partner n/a $2.5M ($1M - $4.0M) ', ' San Francisco (CA) Developer Tools , E-commerce , Enterprise , Marketplaces , Media/Content , SaaS (Seed) , San Francisco Bay Area  , Security , Social Commerce , Social Networks ']
 San Francisco (CA
['Gigi Levy-Weiss NFX General Partner n/a $1M ($500K - $5.0M) ', ' Tel Aviv (Israel

In [147]:
df = pd.DataFrame(investors, columns=['FName','LName','Target','Min','Max','Focus'])
df[['FName','LName']] = nameList
df['Location'] = locList
df = df[['FName','LName','Target','Min','Max','Location','Focus']]
df.head()

,FName,LName,Target,Min,Max,Location,Focus
0,Jeff,Clavier,$1.2M,$750K,$2.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","Palo Alto (CA) , San Francisco (CA) , SF Bay ..."
1,Satya,Patel,$1M,$200K,$2.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","Latin America, Canada , United States, San Fr..."
2,Dave,Samuel,$2.5M,$1M,$4.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","San Francisco (CA) Developer Tools , E-commer..."
3,Gigi,Levy-Weiss,$1M,$500K,$5.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","Tel Aviv (Israel) , Palo Alto (CA) AI , AI , ..."
4,Mike,Maples,$2.6M,$100K,$5.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","San Francisco (CA) , Palo Alto (CA) Develope..."


In [56]:
socialDf = pd.DataFrame(masterSocialList)
socialDf.to_csv('social'+datestr+'.csv',index=False)

In [151]:
df.join(socialDf)

,FName,LName,Target,Min,Max,Location,Focus,0,1,2,3,4
0,Jeff,Clavier,$1.2M,$750K,$2.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","Palo Alto (CA) , San Francisco (CA) , SF Bay ...",https://www.uncorkcapital.com/team/jeff-clavier/,https://www.linkedin.com/in/jeffclavier/,https://www.twitter.com/jeff?lang=en,https://www.angel.co/jeff,https://www.crunchbase.com/person/jeff-clavier
1,Satya,Patel,$1M,$200K,$2.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","Latin America, Canada , United States, San Fr...",https://www.homebrew.co/team,https://www.linkedin.com/in/satyapatel/,https://www.twitter.com/satyap,https://www.angel.co/satyap,https://www.crunchbase.com/person/satya-patel
2,Dave,Samuel,$2.5M,$1M,$4.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","San Francisco (CA) Developer Tools , E-commer...",https://www.freestyle.vc/team_members/dave-sam...,https://www.linkedin.com/in/thedjdave/,https://www.twitter.com/dsamuel,https://www.angel.co/dsamuel,https://www.crunchbase.com/person/david-samuel
3,Gigi,Levy-Weiss,$1M,$500K,$5.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","Tel Aviv (Israel) , Palo Alto (CA) AI , AI , ...",https://www.nfx.com/team/gigi-levy-weiss,https://www.linkedin.com/in/gigilevy/,https://www.twitter.com/GigiLevy,https://www.angel.co/gigi-levy,https://www.crunchbase.com/person/gigi-levy-weiss
4,Mike,Maples,$2.6M,$100K,$5.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","San Francisco (CA) , Palo Alto (CA) Develope...",https://www.floodgate.com/team/mike-maples-jr,https://www.linkedin.com/in/maples/,https://www.twitter.com/m2jr,https://www.angel.co/m2jr,https://www.crunchbase.com/person/mike-maples-jr
...,...,...,...,...,...,...,...,...,...,...,...,...
969,Frederic,Halley,$1.5M,$100K,$5.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","London AI , Enterprise Applications , Enterpr...",https://www.linkedin.com/in/miriccio/,https://www.angel.co/p/michael-riccio,https://www.crunchbase.com/person/michael-riccio,None,None
970,Ramon,Vigdor,$1.5M,$100K,$5.0M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","Amsterdam (Netherlands) AI , SaaS (Seed) Save...",https://www.techstars.com/accelerators/sportst...,https://www.linkedin.com/in/todd-deacon-8391564/,https://www.crunchbase.com/person/todd-deacon,None,None
971,Alex,Samoylenko,$750K,$500K,$1M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","Cyprus Advertising , Advertising , AI , AI , ...",https://www.startershub.com/who-we-are/,https://www.linkedin.com/in/cigdemtoraman/,https://www.angel.co/u/cigdem-toraman,https://www.crunchbase.com/person/cigdem-toraman,None
972,Todd,Breeden,$500K,$250K,$1M,"Palo Alto (CA) , San Francisco (CA) , SF Bay A...","New York (NY) Enterprise , Enterprise , Marke...",https://www.galatabusinessangels.com/en/we/ata...,https://www.linkedin.com/in/ata-uzunhasan-1b26...,https://www.twitter.com/AtaUzunhasan,https://www.angel.co/u/ata-uzunhasan,https://www.crunchbase.com/person/ata-uzunhasan-3


In [152]:
df.to_csv('df'+datestr+'.csv',index=False)

In [106]:
matchSocials = []

for i in range(0,len(df['LName'])):
    iden = df['LName'][i].split("-")[0].lower()
    match = False
    for j in range(0, len(masterSocialList[i])):
        try:
            match = iden in masterSocialList[i][j]
            break;
        except:
            break;
    if match:
        matchSocials.append(masterSocialList[i])
    else:
        matchSocials.append('')
        

In [149]:
realSocials = []

for i in range(0,len(df['LName'])):
    iden = df['LName'][i].split("-")[0].lower()
    match = False
    for j in range(0, len(masterSocialList[i])):
        try:
            match = iden in masterSocialList[i][j]
        except:
            break;
    if match:
        realSocials.append(masterSocialList[i])
    else:
        realSocials.append('')

['https://www.uncorkcapital.com/team/jeff-clavier/', 'https://www.linkedin.com/in/jeffclavier/', 'https://www.crunchbase.com/person/jeff-clavier']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

[]
[]

[]
[]

[]
[]
[]
[]
[]
[]

[]


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

[]
[]

[]

[]
[]

[]
[]

[]
[]
[]


[]
[]
[]

[]

[]

[]
[]
[]


[]
[]
[]
[]
[]
[]

[]
[]
[]

[]
[]
[]
[]
[]

[]






[]
[]

[]
[]


[]
[]

[]
[]
[]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

[]
[]
[]

[]


[]
[]
[]
[]

[]

[]
[]
[]
[]

[]
[]

[]



[]

[]
[]





[]
[]




[]
[]


[]


[]
[]
[]
[]

[]
[]
[]

[]


[]

[]
[]
[]
[]



[]
[]

[]
[]

[]


[]

[]
[]
[]




[]





[]
[]
[]
[]

[]

[]
[]
[]

[]
[]
[]
[]


[]




[]
[]
[]

[]



[]


[]
[]




[]
[]





[]
[]



[]

[]
[]
[]


[]


[]


[]

[]







[]
[]
[]
[]


[]


[]
[]
[]
[]

[]




[]

[]
[]



[]

[]

[]
[]
[]
[]


[]




[]


[]


[]

[]
[]
[]

[]
[]
[]

[]
[]
[]
[]


[]
[]

[]



[]


[]


[]

[]

[]



[]

[]
[]


[]
[]



[]

[]


[]
[]


[]


[]
[]


[]





[]


In [84]:
for j in range(0, len(masterSocialList[0])):
    print('clavier' in masterSocialList[0][j])

True
True
False
False
True


In [88]:
masterSocialList

[['https://www.uncorkcapital.com/team/jeff-clavier/',
  'https://www.linkedin.com/in/jeffclavier/',
  'https://www.twitter.com/jeff?lang=en',
  'https://www.angel.co/jeff',
  'https://www.crunchbase.com/person/jeff-clavier'],
 ['https://www.homebrew.co/team',
  'https://www.linkedin.com/in/satyapatel/',
  'https://www.twitter.com/satyap',
  'https://www.angel.co/satyap',
  'https://www.crunchbase.com/person/satya-patel'],
 ['https://www.freestyle.vc/team_members/dave-samuel/',
  'https://www.linkedin.com/in/thedjdave/',
  'https://www.twitter.com/dsamuel',
  'https://www.angel.co/dsamuel',
  'https://www.crunchbase.com/person/david-samuel'],
 ['https://www.nfx.com/team/gigi-levy-weiss',
  'https://www.linkedin.com/in/gigilevy/',
  'https://www.twitter.com/GigiLevy',
  'https://www.angel.co/gigi-levy',
  'https://www.crunchbase.com/person/gigi-levy-weiss'],
 ['https://www.floodgate.com/team/mike-maples-jr',
  'https://www.linkedin.com/in/maples/',
  'https://www.twitter.com/m2jr',
  'ht

In [107]:
matchSocials

[['https://www.uncorkcapital.com/team/jeff-clavier/',
  'https://www.linkedin.com/in/jeffclavier/',
  'https://www.twitter.com/jeff?lang=en',
  'https://www.angel.co/jeff',
  'https://www.crunchbase.com/person/jeff-clavier'],
 '',
 ['https://www.freestyle.vc/team_members/dave-samuel/',
  'https://www.linkedin.com/in/thedjdave/',
  'https://www.twitter.com/dsamuel',
  'https://www.angel.co/dsamuel',
  'https://www.crunchbase.com/person/david-samuel'],
 ['https://www.nfx.com/team/gigi-levy-weiss',
  'https://www.linkedin.com/in/gigilevy/',
  'https://www.twitter.com/GigiLevy',
  'https://www.angel.co/gigi-levy',
  'https://www.crunchbase.com/person/gigi-levy-weiss'],
 ['https://www.floodgate.com/team/mike-maples-jr',
  'https://www.linkedin.com/in/maples/',
  'https://www.twitter.com/m2jr',
  'https://www.angel.co/m2jr',
  'https://www.crunchbase.com/person/mike-maples-jr'],
 ['https://www.nfx.com/team/morgan-beller-2/',
  'https://www.linkedin.com/in/morganbeller/',
  'https://www.twitt